In [ ]:
!pip install tensorflow-addons
!pip install pyyaml h5py
!pip install toolz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
%tensorflow_version 2.x
import tensorflow as tf
print("Tensorflow version " + tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Tensorflow version 2.9.2


## Setup

In [ ]:
import os
# os.environ["MODIN_CPUS"] = "8"
# from distributed import Client
# client = Client()
import numpy as np
import math
import re
import random
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras.backend as K
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow_addons as tfa
from sklearn import metrics
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import constraints
from tensorflow.keras import initializers
from tensorflow.keras import regularizers
from tensorflow.keras.applications import efficientnet as efn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from tensorflow.keras.constraints import Constraint
# import allel
from scipy.spatial.distance import squareform
%matplotlib inline   
from toolz import interleave
from tqdm import tqdm
from matplotlib import pyplot as plt
import tensorflow_datasets as tfds
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LassoCV, ElasticNetCV
from sklearn.model_selection import KFold,StratifiedKFold

print("Tensorflow version " + tf.__version__)
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.config.experimental_connect_to_cluster(resolver)
# # This is the TPU initialization code that has to be at the beginning.
# tf.tpu.experimental.initialize_tpu_system(resolver)
# print("All devices: ", tf.config.list_logical_devices('TPU'))
# strategy = tf.distribute.TPUStrategy(resolver)

Tensorflow version 2.9.2


## Hardware Config

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    TPU = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', TPU.master())
except ValueError:
    print('Running on GPU')
    TPU = None

if TPU:
    tf.config.experimental_connect_to_cluster(TPU)
    tf.tpu.experimental.initialize_tpu_system(TPU)
    strategy = tf.distribute.TPUStrategy(TPU)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

N_REPLICAS = strategy.num_replicas_in_sync
# Number of computing cores, is 8 for a TPU V3-8
print(f'N_REPLICAS: {N_REPLICAS}')

Running on TPU  grpc://10.10.71.138:8470
N_REPLICAS: 8


## Prepare the data

In [ ]:
# load genotype
genotypes = pd.read_csv('[path]/genotype_full.txt', sep='\t', index_col=0)
genotypes[genotypes == -1] = 0

headers = genotypes.columns[:]

In [ ]:
genotypes.head()

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
01_01,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_02,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
01_03,0,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1
01_04,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
01_06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
X = genotypes
X.shape

(4390, 28220)

In [ ]:
# hyperparameters
feature_size = X.shape[1]
inChannel = 3
learning_rate = 0.01
weight_decay = 0.00001
embed_dim = 64  # Embedding size for each token
num_heads = 8 # Number of attention heads
ff_dim = 32  # Hidden layer size in feed forward network inside transformer
regularization_coef_l1 = 1e-4
batch_size = 20
dropout_rate = 0.25

## Convert to tensorflow dataset

In [ ]:
@tf.function()
def add_attention_mask(X_sample, y_sample):
  depth = 3
  mask_size = tf.cast(X_sample.shape[0]*0.5, dtype=tf.int64)
  mask_idx = tf.reshape(tf.random.shuffle(tf.range(X_sample.shape[0]))[:mask_size], (-1, 1))
  updates = tf.math.add(tf.ones(shape=(mask_idx.shape[0]), dtype=tf.int64), 1)
  X_masked = tf.tensor_scatter_nd_update(X_sample, mask_idx, updates)
  return tf.one_hot(X_masked, depth), tf.one_hot(y_sample, depth-1)

In [ ]:
def map_values_1(x):
  return 0 if (x == 0 or x == 1) else 1

def map_values_2(x):
  return 0 if (x == 0 or x == 2) else 1

def get_dataset(x, chunk_start, chunk_end, start_offset, end_offset, batch_size, training=True):
  AUTO = tf.data.AUTOTUNE

  new_chunk_end = x.shape[1]

  dataset = tf.data.Dataset.from_tensor_slices((x,
                                                x[:, start_offset:new_chunk_end-end_offset]))
  
  if training:
    dataset = dataset.shuffle(x.shape[0], reshuffle_each_iteration=True)
    dataset = dataset.repeat()
  
  # Add Attention Mask
  dataset = dataset.map(add_attention_mask, num_parallel_calls=AUTO, deterministic=False)

  # Prefetech to not map the whole dataset
  dataset = dataset.prefetch(AUTO)

  dataset = dataset.batch(batch_size, drop_remainder=True, num_parallel_calls=AUTO)

  return dataset

## Hyper-params


In [ ]:
kr = 1e-5
drop_prec = 0.25

## Model

In [ ]:
def createSCDA():
  inputt = layers.Input(shape=(feature_size, inChannel))
  # encoder
  x = layers.Conv1D(32, 5, padding='same',activation='relu',
                         kernel_regularizer=regularizers.l1(kr),
                         input_shape=(feature_size, inChannel))(inputt)
  x = layers.MaxPooling1D(pool_size=2)(x)
  x = layers.Dropout(drop_prec)(x)
            
  x = layers.Conv1D(64, 5, padding='same', activation='relu',
                         kernel_regularizer=regularizers.l1(kr)
                         )(x) 
  x = layers.MaxPooling1D(pool_size=2)(x) 
  x = layers.Dropout(drop_prec)(x)

  # bridge
  x = layers.Conv1D(128, 5, padding='same', activation='relu',
                         kernel_regularizer=regularizers.l1(kr)
                         )(x)

  # decoder
  x = layers.Conv1D(64, 5, padding='same', activation='relu',
                         kernel_regularizer=regularizers.l1(kr)
                         )(x) 
  x = layers.UpSampling1D(2)(x) 
  x = layers.Dropout(drop_prec)(x)
            
  x = layers.Conv1D(32, 5, padding='same', activation='relu',
                         kernel_regularizer=regularizers.l1(kr)
                         )(x) 
  x = layers.UpSampling1D(2)(x)
  x = layers.Dropout(drop_prec)(x)

  x = layers.Conv1D(inChannel-1, 5, activation='softmax', padding='same')(x)

  SCDA = tf.keras.models.Model(inputt, x)

  # SCDA.summary()

  return SCDA

In [ ]:
model = createSCDA()
model.build((1, feature_size, inChannel))
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 28220, 3)]        0         
                                                                 
 conv1d_6 (Conv1D)           (None, 28220, 32)         512       
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 14110, 32)        0         
 1D)                                                             
                                                                 
 dropout_4 (Dropout)         (None, 14110, 32)         0         
                                                                 
 conv1d_7 (Conv1D)           (None, 14110, 64)         10304     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 7055, 64)         0         
 1D)                                                       

In [ ]:
def create_model():
  model =  createSCDA()
  optimizer = tfa.optimizers.LAMB(learning_rate=learning_rate)
  model.compile(optimizer, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=tf.keras.metrics.CategoricalAccuracy())
  return model

In [ ]:
METRIC = "val_categorical_accuracy"

def create_callbacks(kfold=0, metric = METRIC):

    reducelr = tf.keras.callbacks.ReduceLROnPlateau(
        monitor= metric,
        mode='auto',
        factor=0.5,
        patience=3,
        verbose=0
    )

    earlystop = tf.keras.callbacks.EarlyStopping(
        monitor= metric,
        mode='auto',
        patience= 10, 
        verbose=1,
        restore_best_weights=True
    )
    
    callbacks = [
                 reducelr,
                 earlystop
                 ]         
    
    return callbacks

## Training

In [ ]:
# A TPU V3-8 has 8 computing cores, the global batch size will be 1/16 x 8 = 8/128
BATCH_SIZE_BASE = 6
# Training configuration
BATCH_SIZE = BATCH_SIZE_BASE * N_REPLICAS if TPU else 32
BATCH_SIZE

48

In [ ]:
# With constraint
missing_perc = 0.1
# N_SPLITS = 5
# batch_size = 50

N_SPLITS=3
NUM_EPOCHS = 1000
results = None
kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=2022)
fold = 0
_x = X.to_numpy()
for train_index, test_index in kf.split(_x):
  fold += 1
  # if fold < 3:
  #   continue
  accuracies = []
  print(f"Training using fold {fold}")
  
  x_train = _x[train_index]
  test_dataset = _x[test_index]
  x_train, x_valid = train_test_split(x_train, test_size=0.10,
                                      random_state=fold,
                                      shuffle=True,)
  
  steps_per_epoch = x_train.shape[0]//BATCH_SIZE
  validation_steps = x_valid.shape[0]//BATCH_SIZE
  train_dataset = get_dataset(x_train, 0, feature_size, 0, 0, BATCH_SIZE)
  valid_dataset = get_dataset(x_valid, 0, feature_size, 0, 0, BATCH_SIZE, training=False)

  K.clear_session()
  callbacks = create_callbacks()
  with strategy.scope():
    model = create_model()
    history = model.fit(train_dataset, steps_per_epoch=steps_per_epoch, epochs=NUM_EPOCHS, 
            validation_data=valid_dataset,
            validation_steps=validation_steps,
            callbacks=callbacks, verbose=0)
  for missing_perc in [
                        0.01,
                        0.05,
                        0.1,
                        0.2
                        ]:
    save_name = f"[path]/YEAST/SCDA/preds_mixed_mr_{missing_perc}_fold_{fold}_.csv"
    avg_accuracy = []
    preds = []
    true_labels = []
    
    to_save_array = np.zeros((test_dataset.shape[0], test_dataset.shape[1]), dtype=object)
    test_X_missing = np.copy(test_dataset)
    test_X_missing = to_categorical(test_X_missing, 3)
    for i in tqdm(list(range(test_dataset.shape[0])), position=0, leave=True):
      rng = np.random.default_rng(i + fold)
      # Generates missing genotypes
      missing_size = int(missing_perc * test_X_missing.shape[1])
      missing_index = rng.integers(low=0, high=test_X_missing.shape[1],
                                        size=missing_size)
      test_X_missing[i:i+1, missing_index, :] = [0, 0, 1]
      # predict
      predict_onehot = model.predict(test_X_missing[i:i+1, :, :])
      # only care the missing position
      predict_missing_onehot = predict_onehot[0:1, missing_index, :]
      # predict label
      predict_missing = np.argmax(predict_missing_onehot, axis=2)
      
      preds.extend(predict_missing.ravel().tolist())
      
      predict_haplotypes = np.argmax(predict_onehot, axis=2)
      to_save_array[i] = np.where(predict_haplotypes==0, -1, 1)
      # real label
      label_missing_onehot = test_dataset[i:i + 1, missing_index]
      # label_missing = np.argmax(label_missing_onehot, axis=2)
      label_missing = test_dataset[i:i + 1, missing_index]
      true_labels.extend(label_missing.ravel().tolist())
      # accuracy
      correct_prediction = np.equal(predict_missing, label_missing)
      accuracy = np.mean(correct_prediction)
      # print('{}/{}, accuracy: {:.4f}'.format(
      #     i, test_X_missing.shape[0], accuracy))

      avg_accuracy.append(accuracy)

    df = pd.DataFrame(to_save_array, columns= headers[:], index = test_index)
    df.to_csv(save_name)
    print('The average imputation accuracy' \
          'on test data with {} missing genotypes is {:.4f}: '
        .format(missing_perc, np.mean(avg_accuracy)))
    cnf_matrix = confusion_matrix(true_labels, preds)
    FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)
    FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
    TP = np.diag(cnf_matrix)
    TN = cnf_matrix.sum() - (FP + FN + TP)
    FP = FP.astype(float)
    FN = FN.astype(float)
    TP = TP.astype(float)
    TN = TN.astype(float)
    # Sensitivity, hit rate, recall, or true positive rate
    TPR = TP/(TP+FN)
    # Specificity or true negative rate
    TNR = TN/(TN+FP)
    print(f"Sensitivity: {np.mean(TPR)}")
    print(f"Specificity: {np.mean(TNR)}")
    print(f"F1-score macro: {f1_score(true_labels, preds, average='macro')}")
    print(f"F1-score micro: {f1_score(true_labels, preds, average='micro')}")
    accuracies.append(np.mean(avg_accuracy))
      
    